# Welcome to the Synergy Single Frame with Image Streamer POC
Created by Fredrik Tärnell - 2019-02-29<br>

Jupyter Notebooks can be found: https://github.com/frippe75/synergy-jupyter-notebooks<br>
Together with a Docker image with all dependencies inluded and a quickstart guide.


### API information

http://h17007.www1.hpe.com/docs/enterprise/servers/oneview4.1/cic-api/en/index.html <br>


### Python specific
https://hewlettpackard.github.io/python-hpOneView/index.html<br>
https://github.com/HewlettPackard/python-hpOneView<br>

## Table of content

- **[Dependencies and logging in](#Dependencies)**<br>
   + [Import the Python OneView library](#Import)<br>
   + [YAML file or embedded document](#YAML)<br>
   + [Login against Oneview](#Login)<br>
   
- **[Settings](#Setting)**<br>
   + [Addresses and Indentifiers](#Address-identifiers)<br>
   + [Networks](#Networks)<br>

- **[Storage](#Storage)**<br>
   + [SAN MAnagers](#SANMgr)<br> 
   + [Storage Systems](#StorageSystems)<br> 
   


  <br> 
- **[Server Profiles](#SP)**<br>
   + [Power On/Off all Profiles](#SP-power)<br>
   + [Delete all Profiles](#SP-delete)<br>
- **[Server Profiles Templates](#SPT)**<br>
   + [Delete all Profiles Templates](#SPT-delete)<br>
- **[Appliances](#Appliances)**<br>
   + [eFuse Composer / Image Streamer](#Appliance-efuse)<br>

<br><br><br>
# Dependencies and logging in



### Import the Python OneView library with some additional dependencies for this playbook <a name="Import"></a>

In [4]:
from hpOneView.oneview_client import OneViewClient
from hpOneView.exceptions import HPOneViewException
import pandas, yaml
from IPython.display import display, Markdown
from pprint import pprint

poc_file = None

# Simple helper function to make nicer tables using markdown and the pandas lib
# TODO: get it into helper module that gets imported above 
def my_table(source=dict(), resource="Resource", columns='name'):
    data = pandas.DataFrame.from_dict(source)
    table = data.to_html(index=False,header=True,columns=columns)
    
    display(Markdown('### ' + resource + ' in Synergy Composer at https://' + config['ip']))
    display(Markdown(table))

### Configure your environment
**Please note:** that if you use the option below to get the YAML doc from a file the config dict will be overwritten by the config from the file. During the login, check that you are logged in against the correct Oneview instance

In [31]:
composer_ip = "synergy.hpedemo.local"
config = {
    "api_version": "800",
    "ip": composer_ip,
    "credentials": {
        "userName": "Administrator",
        "authLoginDomain": "local",
        "password": "HPEc0nverged!"
    }
}

### YAML file or embedded document <a name="YAML"></a>
Determine wheter to load the YAML doc from a variable or file<br>
Setting the filename takes precedence over variable

In [37]:
# Comment out to use the poc_file to allow the embedded poc_doc to be used
poc_file = "synergy420-DCS-poc.yaml"

# Example YAML stucture for allowing Addresses and Indentifiers part work without a file
poc_doc = """
---
  addresses:
    # Synergy Management 
    - networkId: 10.48.0.0
      subnetmask: 255.255.255.0
      gateway: 10.48.0.1
      domain: hpedemo.local
      dnsServers: 
        - 10.48.10.4
        - 10.48.10.6
      range:
        name: LE-POC-01
        startAddress: 10.48.0.104
        endAddress: 10.48.0.124
    # Image Streamer iSCSI 
    - networkId: 192.168.10.0
      subnetmask: 255.255.255.0
      range:
        name: iSCSI-boot
        startAddress: 192.168.10.10
        endAddress: 192.168.10.100
    # VMware Mgmt 
    - networkId: 10.48.10.0
      subnetmask: 255.255.255.0
      gateway: 10.48.10.1
      domain: hpedemo.local
      dnsServers: 
        - 10.48.10.4
        - 10.48.10.6
      range:
        name: LE-POC-01
        startAddress: 10.48.10.70
        endAddress: 10.48.10.79
    # vMotion 
    - networkId: 192.168.20.0
      subnetmask: 255.255.255.0
      range:
        name: vMotion
        startAddress: 192.168.20.10
        endAddress: 192.168.20.100
"""
# load/parse the YAML doc into a variable
if poc_file is not None:
    with open(poc_file) as f:
        poc_doc = f.read()
    # print(poc_doc)
    poc = yaml.load(poc_doc)
    # Check if Oneview config key is part of the YAML file and use that for logging in.
    if 'config' in poc:
        print("\nOneview endpoint config was found and is used. Removing it to avoid displaying passwords\n")
        config = poc.pop('config', None)
else:
    poc = yaml.load(poc_doc)
    
# Print it out for sanitycheck
print (yaml.dump(poc))


Oneview endpoint config was found and is used. Removing it to avoid displaying passwords

SANManagers:
- {authProtocol: MD5, ip: 172.18.20.1, password: dcsdcsdcs, securityLevel: Authentication,
  type: Cisco, userName: dcs-MD5}
- {authProtocol: MD5, ip: 172.18.20.2, password: dcsdcsdcs, securityLevel: Authentication,
  type: Cisco, userName: dcs-MD5}
addresses:
- dnsServers: [10.48.10.4, 10.48.10.6]
  domain: hpedemo.local
  gateway: 10.48.0.1
  networkId: 10.48.0.0
  range: {endAddress: 10.48.0.124, name: LE-POC-01, startAddress: 10.48.0.104}
  subnetmask: 255.255.255.0
- networkId: 192.168.10.0
  range: {endAddress: 192.168.10.100, name: iSCSI-boot, startAddress: 192.168.10.10}
  subnetmask: 255.255.255.0
- dnsServers: [10.48.10.4, 10.48.10.6]
  domain: hpedemo.local
  gateway: 10.48.10.1
  networkId: 10.48.10.0
  range: {endAddress: 10.48.10.79, name: LE-POC-01, startAddress: 10.48.10.70}
  subnetmask: 255.255.255.0
- networkId: 192.168.20.0
  range: {endAddress: 192.168.20.100, na

<br><br>
## Login against the Appliance / Composer instance: <a name="Login"></a>
Create new object "ov" with the config (see above) using the JSON config dict above

In [33]:
try:
    ov = OneViewClient(config)
    print("\nLogged in against " + config['ip'] + " successfully\n")
except HPOneViewException as e:
    print("\nLogged in failed:")
    print(e.msg)


Logged in against syn420-dcs.hpedemo.local successfully



<br><br>
# Settings  <a name="Settings"></a>
## Addresses and Indentifiers  <a name="Address-identifiers"></a>
Here we define required networks for both Synergy Management network as well as Image Streamer iSCSi boot network and VMware vMotion.

In [27]:
# Need to reload/parse the YAML doc again since the options.pop mutates the existing dictionary so the contents of the dictionary changes 
poc = yaml.load(poc_doc)

# Loop over the addresses array and create subnets and ranges
if 'addresses' in poc:
    for options in poc['addresses']:

        options.update({
            "type": "Subnet",
            "name": options['networkId']
        })
        # Remove/extract  the range key since it does not belong there.
        range_option = options.pop('range', None)
        #pprint(options)  

        try:
            print("Create IPv4 subnet " + options['networkId'] +" for id pools")
            ipv4_subnet = ov.id_pools_ipv4_subnets.create(options)

            #pprint(range_option)

            range_option.update({
                "type": "Range",
                "rangeCategory": "Custom",
                "subnetUri": ipv4_subnet['uri']
            })
            print("Creating an IPv4 Range for id pools\n")
            ipv4_range = ov.id_pools_ipv4_ranges.create(range_option)
            # pprint(ipv4_range)
        except HPOneViewException as e:
            print(e.msg + "\n")
else:
    print("No Addresses in the poc dict. Nothing created!")    
    
print("\nGetting all IPv4 subnets...")
all_subnets = ov.id_pools_ipv4_subnets.get_all()

#pprint(all_subnets)

my_table(all_subnets, resource="Subnets", columns=['name','networkId','subnetmask','domain','gateway'])

Create IPv4 subnet 10.48.0.0 for id pools
The subnet for Network ID - 10.48.0.0 and Subnet Mask - 255.255.255.0 is overlapping with the subnet for Network ID - 10.48.0.0 and Subnet Mask - 255.255.255.0.

Create IPv4 subnet 192.168.10.0 for id pools
The subnet for Network ID - 192.168.10.0 and Subnet Mask - 255.255.255.0 is overlapping with the subnet for Network ID - 192.168.10.0 and Subnet Mask - 255.255.255.0.

Create IPv4 subnet 10.48.10.0 for id pools
The subnet for Network ID - 10.48.10.0 and Subnet Mask - 255.255.255.0 is overlapping with the subnet for Network ID - 10.48.10.0 and Subnet Mask - 255.255.255.0.

Create IPv4 subnet 192.168.20.0 for id pools
The subnet for Network ID - 192.168.20.0 and Subnet Mask - 255.255.255.0 is overlapping with the subnet for Network ID - 192.168.20.0 and Subnet Mask - 255.255.255.0.


Getting all IPv4 subnets...


### Subnets in Synergy Composer at https://syn420-dcs.hpedemo.local

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>name</th>
      <th>networkId</th>
      <th>subnetmask</th>
      <th>domain</th>
      <th>gateway</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>10.48.0.0</td>
      <td>10.48.0.0</td>
      <td>255.255.255.0</td>
      <td>hpedemo.local</td>
      <td>10.48.0.1</td>
    </tr>
    <tr>
      <td>10.48.10.0</td>
      <td>10.48.10.0</td>
      <td>255.255.255.0</td>
      <td>hpedemo.local</td>
      <td>10.48.10.1</td>
    </tr>
    <tr>
      <td>192.168.10.0</td>
      <td>192.168.10.0</td>
      <td>255.255.255.0</td>
      <td>None</td>
      <td>None</td>
    </tr>
    <tr>
      <td>192.168.20.0</td>
      <td>192.168.20.0</td>
      <td>255.255.255.0</td>
      <td>None</td>
      <td>None</td>
    </tr>
  </tbody>
</table>

<br><br>
## Networks  <a name="Networks"></a>

In [21]:
# Need to reload/parse the YAML doc again since the options.pop mutates the existing dictionary so the contents of the dictionary changes 
poc = yaml.load(poc_doc)

# My defaults
defaults = {
    "ethernetNetworkType": "Tagged",
    "purpose": "General",
    "smartLink": True,
    "privateNetwork": False,
    "type" : "ethernet-networkV4",
    "connectionTemplateUri": None,
    #"bandwidth": {
    #    "maximumBandwidth": 10000,
    #    "typicalBandwidth": 2000
    #}
}

# Loop over the addresses array and create subnets and ranges
if 'networks' in poc:
    for options in poc['networks']:

        # Remove/extract the association key since it does not belong there.
        association_subnet_id = options.pop('association', None)
        if association_subnet_id is not None:
            # TODO: This uses the get_all which returns an array. Best option is to use get(ID) but that is not implemented 
            #       for resource subnet properly. Doc indicates it should.
            subnet = ov.id_pools_ipv4_subnets.get_all(filter='networkId="' + association_subnet_id + '"')
            subnet_uri = subnet[0]['uri']
            # print(subnet_uri)
            options.update({"subnetUri": subnet_uri})

        # Merge with defaults new style of dictonary unpacking since Python 3.5 https://treyhunner.com/2016/02/how-to-merge-dictionaries-in-python/
        net_options = {**defaults, **options}
        # pprint("\n" + net_options)  

        try:
            print("\nCreating ethernet network " + net_options['name'] )
            ethernet_network = ov.ethernet_networks.create(net_options)
            print("Created succesfully.")
        except HPOneViewException as e:
            print("Error during creation of: " + net_options['name'])
            print(e.msg)      
else:
    print("No Networks in the poc dict. Nothing created!")
        
print("\nGetting all IPv4 networks...")
all_nets = ov.ethernet_networks.get_all()

#pprint(all_nets)        
my_table(all_nets, resource="Networks", columns=['name','description','vlanId','uri','purpose'])    


Creating ethernet network VLAN2000_10.48.0
Created succesfully.

Creating ethernet network VLAN2000_10.48.10
Created succesfully.

Creating ethernet network ImageStreamer-Boot
Created succesfully.

Creating ethernet network ESX_vMotion_FrameInternal
Created succesfully.

Creating ethernet network Tunnel01
Created succesfully.

Creating ethernet network VLAN400_10.2
Created succesfully.

Creating ethernet network VLAN402_10.3
Created succesfully.

Creating ethernet network NFS-Simplivity
Created succesfully.

Getting all IPv4 networks...


### Networks in Synergy Composer at https://syn420-dcs.hpedemo.local

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>name</th>
      <th>description</th>
      <th>vlanId</th>
      <th>uri</th>
      <th>purpose</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>VLAN2000_10.48.0</td>
      <td>None</td>
      <td>2000</td>
      <td>/rest/ethernet-networks/23c5b8e0-8bfb-4741-9d6...</td>
      <td>Management</td>
    </tr>
    <tr>
      <td>NFS-Simplivity</td>
      <td>None</td>
      <td>2010</td>
      <td>/rest/ethernet-networks/284796fc-f3d9-465d-add...</td>
      <td>General</td>
    </tr>
    <tr>
      <td>ESX_vMotion_FrameInternal</td>
      <td>None</td>
      <td>1000</td>
      <td>/rest/ethernet-networks/35b46e80-ebaf-427c-883...</td>
      <td>ISCSI</td>
    </tr>
    <tr>
      <td>VLAN2000_10.48.10</td>
      <td>None</td>
      <td>2001</td>
      <td>/rest/ethernet-networks/481b00e9-5779-48a8-b09...</td>
      <td>Management</td>
    </tr>
    <tr>
      <td>Tunnel01</td>
      <td>None</td>
      <td>0</td>
      <td>/rest/ethernet-networks/4ec86446-fff3-4b70-a5c...</td>
      <td>General</td>
    </tr>
    <tr>
      <td>VLAN400_10.2</td>
      <td>None</td>
      <td>400</td>
      <td>/rest/ethernet-networks/7c5f4ef3-0176-4221-8a1...</td>
      <td>General</td>
    </tr>
    <tr>
      <td>ImageStreamer-Boot</td>
      <td>None</td>
      <td>1001</td>
      <td>/rest/ethernet-networks/bbdbcbc7-c1cd-4053-a7f...</td>
      <td>ISCSI</td>
    </tr>
    <tr>
      <td>VLAN402_10.3</td>
      <td>None</td>
      <td>300</td>
      <td>/rest/ethernet-networks/c0f7e579-9588-4925-ac7...</td>
      <td>General</td>
    </tr>
  </tbody>
</table>

<br><br>
## SAN Managers  <a name="SANMgr"></a>
This part is pretty much a copy from the pythonOneview examples<br>
https://github.com/HewlettPackard/python-hpOneView/blob/master/examples/san_managers.py<br>
<br>
Difference being it uses the YAML structure to created the listed resources as well as takes care of specifying defaults when needed.


In [29]:
# Need to reload/parse the YAML doc again since the options.pop mutates the existing dictionary so the contents of the dictionary changes 
poc = yaml.load(poc_doc)

# Loop over the addresses array and create subnets and ranges
if 'networks' in poc:
    for mgrs in poc['SANManagers']:

        # Handle Cisco in one way
        if 'type' in mgrs and mgrs['type'] == 'Cisco':
            # print(mgrs['type'])

            # passed values or defaults not optional parameters
            if 'snmpPort' in mgrs:
                snmp_port = mgrs['snmpPort']
            else:
                snmp_port = 161

            if 'authProtocol' in mgrs:
                auth_proto = mgrs['authProtocol']
            else:
                auth_proto = "sha"

            if 'privProtocol' in mgrs:
                priv_proto = mgrs['privProtocol']
            else:
                priv_proto = "aes-128"

            options = {
                'providerDisplayName': 'Cisco',
                'connectionInfo': [
                    {
                        'name': 'Host',
                        'displayName': 'Host',
                        'required': True,
                        'value': mgrs['ip'],
                        'valueFormat': 'IPAddressOrHostname',
                        'valueType': 'String'
                    },{
                        'name': 'SnmpPort',
                        'displayName': 'SnmpPort',
                        'required': True,
                        'value': snmp_port,
                        'valueFormat': 'None',
                        'valueType': 'Integer'
                    },{
                        'name': 'SnmpUserName',
                        'displayName': 'SnmpUserName',
                        'required': True,
                        'value': mgrs['userName'],
                        'valueFormat': 'None',
                        'valueType': 'String'
                    },{
                        'name': 'SnmpAuthLevel',
                        'displayName': 'SnmpAuthLevel',
                        'required': True,
                        'value': 'authnopriv',
                        'valueFormat': 'None',
                        'valueType': 'String'
                    },{
                        'name': 'SnmpAuthProtocol',
                        'displayName': 'SnmpAuthProtocol',
                        'required': False,
                        'value': auth_proto,
                        'valueFormat': 'None',
                        'valueType': 'String'
                    },{
                        'name': 'SnmpAuthString',
                        'displayName': 'SnmpAuthString',
                        'required': False,
                        'value': mgrs['password'],
                        'valueFormat': 'SecuritySensitive',
                        'valueType': 'String'
                    },{
                        'name': 'SnmpPrivProtocol',
                        'displayName': 'SnmpPrivProtocol',
                        'required': False,
                        'value': priv_proto,
                        'valueFormat': 'None',
                        'valueType': 'String'
                    },{
                        'name': 'SnmpPrivString',
                        'displayName': 'SnmpPrivString',
                        'required': False,
                        'value': '',
                        'valueFormat': 'SecuritySensitive',
                        'valueType': 'String'
                    }
                ]
            }

        # Handle Brocade in another way
        if 'type' in mgrs and mgrs['type'] == 'Brocade':   
            # passed values or defaults not optional parameters
            if 'port' in mgrs:
                bna_port = mgrs['port']
            else:
                bna_port = 5989

            if 'UseSsl' in mgrs:
                use_ssl = mgrs['UseSsl']
            else:
                use_ssl = True

            options = {
                'providerDisplayName': 'Brocade',
                'connectionInfo': [{
                    'name': 'Host',
                    'value': mgrs['ip']
                },{
                    'name': 'Port',
                    'value': bna_port
                },{
                    'name': 'Username',
                    'value': mgrs['userName']
                },{
                    'name': 'Password',
                    'value': mgrs['password']
                },{
                    'name': 'UseSsl',
                    'value': use_ssl
                }]
            }

        # Handle HPE San Managers not implemented
        if 'type' in mgrs and mgrs['type'] == 'HPE':
            # TODO: Implement this 
            print("WARNING: HPE San Manager types not implemented.")

        try:
            # print(options)
            provider_uri = ov.san_managers.get_provider_uri(mgrs['type'])
            # print(provider_uri)
            print("Adding SAN Manager " + mgrs['ip'] + "of type " + mgrs['type'])
            san_manager = ov.san_managers.add(options, provider_uri)
            print("Created succesfully.")
        except HPOneViewException as e:
            print("Error during creation of SAN Manager: " + mgrs['ip'])
            print(e.msg) 
else:
    print("No SAN Managers in the poc dict. Nothing created!")
    
print("\nGetting all SAN Managers...")
all_san_mgrs = ov.san_managers.get_all()
#pprint(all_san_mgrs)  

# This will fail for Brocade SAN Managers, TODO: do get_all(filter=Brocade) and handle each type separately
if all_san_mgrs:
    my_table(all_san_mgrs, resource="SAN Managers", columns=['deviceManagerUrl','deviceManagerVersion','description','providerDisplayName','state','status']) 
else:
    print("No SAN Managers found")

Adding SAN Manager 172.18.20.1of type Cisco
Error during creation of SAN Manager: 172.18.20.1
Duplicate SAN manager '172.18.20.1' at '/rest/fc-sans/device-managers/16eb8a22-c055-49d9-996e-aa05009c8518' is already imported.
Adding SAN Manager 172.18.20.2of type Cisco
Error during creation of SAN Manager: 172.18.20.2
Duplicate SAN manager '172.18.20.2' at '/rest/fc-sans/device-managers/43aed419-37d4-43d3-8b67-aa05009c88b5' is already imported.

Getting all SAN Managers...


### SAN Managers in Synergy Composer at https://syn420-dcs.hpedemo.local

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>deviceManagerUrl</th>
      <th>deviceManagerVersion</th>
      <th>description</th>
      <th>providerDisplayName</th>
      <th>state</th>
      <th>status</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>172.18.20.1</td>
      <td>6.2(5)</td>
      <td>MDS 9250i 40 FC 2 IPS 8 FCoE (1 Slot) Chassis</td>
      <td>Cisco</td>
      <td>Managed</td>
      <td>OK</td>
    </tr>
    <tr>
      <td>172.18.20.2</td>
      <td>6.2(5)</td>
      <td>MDS 9250i 40 FC 2 IPS 8 FCoE (1 Slot) Chassis</td>
      <td>Cisco</td>
      <td>Managed</td>
      <td>OK</td>
    </tr>
  </tbody>
</table>

<br><br>
## Storage Systems <a name="StorageSystems"></a>
This will only add the "no Domain" pools and all of them

In [58]:
# Need to reload/parse the YAML doc again since the options.pop mutates the existing dictionary so the contents of the dictionary changes 
poc = yaml.load(poc_doc)

defaults = {
    "family": 'StoreServ'
}

# Loop over the storage systems array and add them to oneview
if 'storageSystems' in poc:
    for array in poc['storageSystems']:
        #pprint (array)
        
        # Merge with defaults new style of dictonary unpacking since Python 3.5 https://treyhunner.com/2016/02/how-to-merge-dictionaries-in-python/
        array_options = {**defaults, **array}
        # pprint("\n" + array_options) 
        
        # Add and update storage system for management
        try:
            print("\nAdding " + storage_system['name'] + " to Oneview")
            storage_system = ov.storage_systems.add(array_options)
            print("Succesfully added it.")
        except HPOneViewException as e:
            storage_system = ov.storage_systems.get_by_hostname(array_options['hostname'])
            if storage_system:
                print("Storage system '%s' was already added.\n   uri = '%s'" %
                      (storage_system['name'], storage_system['uri']))
            else:
                print(e.msg)
        
        # Adds managed domains and managed pools to StoreServ storage systems
        # This is a one-time only action, after this you cannot change the managed values
        if not storage_system['deviceSpecificAttributes']['managedDomain']:
            storage_system['deviceSpecificAttributes']['managedDomain'] = storage_system[
                'deviceSpecificAttributes']['discoveredDomains'][0]
            for pool in storage_system['deviceSpecificAttributes']['discoveredPools']:
                if pool['domain'] == storage_system['deviceSpecificAttributes']['managedDomain']:
                    pool_to_manage = pool
                    storage_system['deviceSpecificAttributes']['discoveredPools'].remove(pool)
                    pprint(pool_to_manage)
                    break
            storage_system['deviceSpecificAttributes']['managedPools'] = [pool_to_manage]
            ov.storage_systems.update(storage_system)
            print("\nUpdated 'managedDomain' to '{}' so storage system can be managed".format(
                  storage_system['deviceSpecificAttributes']['managedDomain']))
            print("\nManaged storage pool '{}' at uri: '{}'".format(storage_system['deviceSpecificAttributes'][
                  'managedPools'][0]['name'], storage_system['deviceSpecificAttributes']['managedPools'][0]['uuid']))

print("\nGetting all storage systems...")
all_arrays = ov.storage_systems.get_all()
#pprint(all_arrays)        

my_table(all_arrays, resource="Storage Systems", columns=['displayName','family', 'hostname','supportsFC','supportsIscsi','state'])                    


Adding ThreePAR-1 to Oneview
Storage system 'ThreePAR-1' was already added.
   uri = '/rest/storage-systems/TXQ1000307'
{'deviceType': 'FC',
 'domain': 'NO DOMAIN',
 'freeCapacity': '861140942848',
 'name': 'FC_r1',
 'raidLevel': 'RAID1',
 'totalCapacity': '1099511627776',
 'uuid': '5dd206a8-90ce-4068-bd7a-51c1d29d4e60'}

Updated 'managedDomain' to 'NO DOMAIN' so storage system can be managed

Managed storage pool 'FC_r1' at uri: '5dd206a8-90ce-4068-bd7a-51c1d29d4e60'

Getting all storage systems...


### Storage Systems in Synergy Composer at https://syn420-dcs.hpedemo.local

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>displayName</th>
      <th>family</th>
      <th>hostname</th>
      <th>supportsFC</th>
      <th>supportsIscsi</th>
      <th>state</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>ThreePAR-1</td>
      <td>StoreServ</td>
      <td>172.18.11.11</td>
      <td>True</td>
      <td>False</td>
      <td>Managed</td>
    </tr>
  </tbody>
</table>

<br><br>
# Storage operations
## Show configured / existing storage (systems and pools)

In [38]:
storage_systems = ov.storage_systems.get_all()
storage_pools = ov.storage_pools.get_all()

# storage_pool_name = 'FC_r1'
# storage_pools = oneview_client.storage_pools.get_by('name', storage_pool_name)[0]

print("Storage Systems:")
if storage_systems:
    for stor in storage_systems: 
        print(stor['displayName'] + "    URI: " + stor['uri'])
else:
    print("None defined!")
    
print("\nStorage Pools:")
if storage_pools:
    for storpool in storage_pools:
        print(storpool['name'] + "    URI: " + stor['uri'])
else:
    print("None defined!")

Storage Systems:
None defined!

Storage Pools:
None defined!


In [57]:
if storpool is not None: 
    print (storpool)

NameError: name 'storpool' is not defined

<br><br>
## Add Storage Systems

<br><br>
## Show volume templates

In [31]:
storage_volume = ov.storage_volume_templates.get_all(filter="\"isRoot='False'\"")
if storage_volume:
    for storvol in storage_volume:
        print(storvol['name'])
else:
    print ("No storage volumes exists!")

No storage volumes exists!


<br><br>
## Create volume template

In [13]:
storage_pool_name = 'FC_r5'

# Get the storage pool by name to use in options
storage_pool = ov.storage_pools.get_by('name', storage_pool_name)[0]

# Gets the first Root Storage Volume Template available to use in options
root_template = ov.storage_volume_templates.get_all(filter="\"isRoot='True'\"")[0]
print(root_template['uri'])

options = {
    "name": "First Volume Template",
    "description": "",
    "rootTemplateUri": root_template['uri'],
    "properties": {
        "name": {
            "meta": {
                "locked": False
            },
            "type": "string",
            "title": "Volume name",
            "required": True,
            "maxLength": 100,
            "minLength": 1,
            "description": "A volume name between 1 and 100 characters"
        },
        "size": {
            "meta": {
                "locked": False,
                "semanticType": "capacity"
            },
            "type": "integer",
            "title": "Capacity",
            "default": 1073741824,
            "maximum": 17592186044416,
            "minimum": 268435456,
            "required": True,
            "description": "The capacity of the volume in bytes"
        },
        "description": {
            "meta": {
                "locked": False
            },
            "type": "string",
            "title": "Description",
            "default": "",
            "maxLength": 2000,
            "minLength": 0,
            "description": "A description for the volume"
        },
        "isShareable": {
            "meta": {
                "locked": False
            },
            "type": "boolean",
            "title": "Is Shareable",
            "default": False,
            "description": "The shareability of the volume"
        },
        "storagePool": {
            "meta": {
                "locked": False,
                "createOnly": True,
                "semanticType": "device-storage-pool"
            },
            "type": "string",
            "title": "Storage Pool",
            "format": "x-uri-reference",
            "required": True,
            "description": "A common provisioning group URI reference",
            "default": storage_pool['uri']
        },
        "snapshotPool": {
            "meta": {
                "locked": True,
                "semanticType": "device-snapshot-storage-pool"
            },
            "type": "string",
            "title": "Snapshot Pool",
            "format": "x-uri-reference",
            "default": storage_pool['uri'],
            "description": "A URI reference to the common provisioning group used to create snapshots"
        },
        "provisioningType": {
            "enum": [
                "Thin",
                "Full",
                "Thin Deduplication"
            ],
            "meta": {
                "locked": True,
                "createOnly": True
            },
            "type": "string",
            "title": "Provisioning Type",
            "default": "Thin",
            "description": "The provisioning type for the volume"
        }
    }
}

volume_template = oneview_client.storage_volume_templates.create(options)
print("Storage volume created...")

/rest/storage-volume-templates/bf62b10a-7a6c-4c7a-ba1d-a8c800809aaa
Storage volume created...


## Delete storage template
""" don´t use it, for the demo.. """

In [14]:
storage_volume = oneview_client.storage_volume_templates.get_all(filter="\"name='Roundtable Volume Template'\"")
for storvol in storage_volume:
    oneview_client.storage_volume_templates.delete(storvol)
    

## Show Enclosure / Server Hardware / Bay

In [15]:
print ("show enclosure group (enclosure_group_uri)")
enclosure = oneview_client.enclosure_groups.get_all()
for enc in enclosure:
    print(enc['name'] + " " + enc['uri'])

print("\nshow server hardware (server_hardware_type_uri)")
server_hardware_types = oneview_client.server_hardware_types.get_all(sort='name:descending')
# print(server_hardware_types)
for serverhw in server_hardware_types:
    # print(' %s ' % serverhw['model'])
    print(serverhw['model'] + " URI: " + serverhw['uri'] )

print("\nEnclosure/Bay (server_hardware_uri)")   
server_hardware = oneview_client.server_hardware.get_all()
for server in server_hardware:
    print(server['name'] + " " + server['model'] + " " + server['uri'])

show enclosure group (enclosure_group_uri)
SY 3 Frame SAS FC VC /rest/enclosure-groups/73668dd8-d2cf-4004-b7df-8ca7cfeabbc0
SY 1 Frame VC /rest/enclosure-groups/92e527ff-7129-489f-87dc-e7d5821f13c6
SY 3 Frame SAS VC /rest/enclosure-groups/109eed5b-4cff-470e-b2ca-ffb26eb5c678

show server hardware (server_hardware_type_uri)
Synergy 660 Gen9 URI: /rest/server-hardware-types/A435F2DD-9EB9-4F98-B854-1A6B35140DC9
Synergy 660 Gen10 URI: /rest/server-hardware-types/7AA9D8EE-C46A-4901-8D10-C252AF9DB1A4
Synergy 480 Gen9 URI: /rest/server-hardware-types/860CAF19-0D67-401A-A1D2-536AA9356102
Synergy 480 Gen9 URI: /rest/server-hardware-types/1131E4D0-9C51-4615-A20E-CCF00F02A94F
Synergy 480 Gen10 URI: /rest/server-hardware-types/77E175FB-A4AA-4E95-8780-D1E9A40D3887
Synergy 480 Gen10 URI: /rest/server-hardware-types/56C7D5F1-6221-4ACB-938B-21916B7DE5F3

Enclosure/Bay (server_hardware_uri)
CTC Synergy HE01, bay 1 Synergy 480 Gen10 /rest/server-hardware/39313738-3134-5A43-4A37-343030373531
CTC Synergy 

<br>
## Show Serverprofile


In [16]:
print("show server profile templates:")
all_srv_templates = oneview_client.server_profile_templates.get_all()
for srv_tmp in all_srv_templates:
    print(srv_tmp['name'] + "   URI:   " + srv_tmp['uri'])

print("\nshow server profiles:")
all_profiles = oneview_client.server_profiles.get_all()
for profile in all_profiles:
    print(profile['name']+ "   URI:  " + srv_tmp['uri'])

# my_profile = oneview_client.server_profiles.get_by_name("Roundtable - API Demo Template (DirkD)")

show server profile templates:
SRT-Storage-BfS-W2k16   URI:   /rest/server-profile-templates/1df43d4a-76c3-4821-bff3-729446dc7231
Roundtable - Python (DirkD)   URI:   /rest/server-profile-templates/1e2c29f5-e146-42d4-9f78-a057b41d3953
SRT-LC-WIN-TP   URI:   /rest/server-profile-templates/308e1b0a-2e7c-40aa-bdf9-84eb87b42548
SRT-StorageDemo-ESX   URI:   /rest/server-profile-templates/534d2abd-989c-456e-a84b-8c112ce3fcac
Demo-CTC Windows2016-DC-byI3s   URI:   /rest/server-profile-templates/7a5d77bc-a372-4a44-8a52-b538ba703b97
SRT-Storage-BfS-ESX   URI:   /rest/server-profile-templates/823883ee-0f0e-4554-8e00-a909001b28af
Demo-CTC Windows2016-DC-byI3s_Gen10   URI:   /rest/server-profile-templates/894f818a-9604-48cb-ab9f-ea6c240a9381
DEMO-CTC ESXi Server Template   URI:   /rest/server-profile-templates/ac2995c1-8b21-482b-80e8-7157f1984b75
SRT-LC-WIN BfS   URI:   /rest/server-profile-templates/b18b3cf9-8f47-4a6a-be35-18bba3078acd
DEMO-CTC Stratoscale Template   URI:   /rest/server-profile-t

<br><br>
## Create server profile (takes 2-3 minutes)

In [17]:
powerOn = {
    "powerState": "On",
    "powerControl": "MomentaryPress"
}

powerOff = {
    "powerState": "Off",
    "powerControl": "PressAndHold"
}

#template_name = "Roundtable - API Demo Template (DirkD)"
template_name = "Roundtable - Python (DirkD)"
server_name = "Roundtable - API Demo Server (DirkD)"

# server_hardware_uri = '/rest/server-hardware/39313738-3133-5A43-4A37-343030373931' # SY660 G10
server_hardware_uri = '/rest/server-hardware/39313738-3134-5A43-4A37-343030373536' # SY480
server_template_uri = oneview_client.server_profile_templates.get_by_name(template_name)

server_power = oneview_client.server_hardware.update_power_state(powerOff, server_hardware_uri) # turn off server

try:
    print ("create server profile")
    roundtable_server = oneview_client.server_profile_templates.get_new_profile(server_template_uri['uri'])
    roundtable_server["name"] = server_name
    roundtable_server["serverHardwareUri"] = server_hardware_uri

    for name in roundtable_server["osDeploymentSettings"]["osCustomAttributes"]:
        if (name['name'] == "NewUser"):
            name['value']= "dirk"
        if (name['name'] == "NewUserPassword"):
            name['value']= "HalloRoundtable!"   
    profile = oneview_client.server_profiles.create(roundtable_server)

    
except:
    print(server_name + " Server already exists")

server_power = oneview_client.server_hardware.update_power_state(powerOn, server_hardware_uri) # turn on server


create server profile


### Show individual parameters from Image (Imagestreamer)

In [34]:
    #print ("create server profile")
    my_server = ov.server_profile_templates.get_new_profile(server_template_uri['uri'])
    my_server["name"] = server_name
    my_server["serverHardwareUri"] = server_hardware_uri

    for name in my_server["osDeploymentSettings"]["osCustomAttributes"]:
        print(name['name'] + ": " + str(name['value']))
   


NameError: name 'server_template_uri' is not defined

## Delete server profile
!!! do not use, if not necessary !!!

In [ ]:
server_power = oneview_client.server_hardware.update_power_state(powerOff, server_hardware_uri) # turn on server
oneview_client.server_profiles.delete(profile)

### example: how to get the values from json

In [ ]:
    server_hardware = oneview_client.server_hardware.get_all()
   # print(server_hardware)
    
    for server in server_hardware:
        for ports in server['portMap']['deviceSlots']:
            for mac in ports['physicalPorts']: 
                for wwnn in mac['virtualPorts']:
                    print(server['name']+ " model:" + server['model'] + " " + str(server['memoryMb']) + " MB mac: " + str(mac['mac']) + " wwnn: " + str(wwnn['wwnn'])) 

## Application deployment


Webserver NGINX running on docker.

##### What we need:

IP address of deployed system.

<br>
<img src="Pictures/NGINX.png" height="500" width="500" align="left">


In [21]:
import os
import paramiko
import time

ssh = paramiko.SSHClient()

server_name = "Roundtable - API Demo Server (DirkD)"
username = 'root'
password = 'Compaq1!'

deployed_server = oneview_client.server_profiles.get_by_name(server_name)

for name in deployed_server["osDeploymentSettings"]["osCustomAttributes"]:
    if (name['name'] == "Team0NIC1.ipaddress"):
        ip_address = name['value']

        
print("Login with user: " + username + " Server:" + ip_address)        
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())         # add unknown Host-Keys
ssh.connect(ip_address, username=username, password=password)     # login
ssh.exec_command('docker run -d --name nginx -p 80:80 nginx')
#time.sleep(10)
stdin, stdout, stderr = ssh.exec_command("docker exec -it nginx sed -i '\''s/nginx/the Synergy Roundtable/g'\'' /usr/share/nginx/html/index.html", get_pty=True)
print("http://" + ip_address)
# print(stdout.read())
# print(stderr.read())


Login with user: root Server:10.0.20.85
http://10.0.20.85


### Excel Export

xlswriter:
https://xlsxwriter.readthedocs.io/

example: we extrace some data to excel XLS

In [32]:
import xlsxwriter
workbook = xlsxwriter.Workbook('roundtable.xlsx')
worksheet = workbook.add_worksheet()

# Add a bold format to use to highlight cells.
bold = workbook.add_format({'bold': True})

# Text with formatting.
worksheet.write(0,0, 'Synergy Roundtable', bold)

# Start from the first cell below the headers.
row = 4
worksheet.write(row, 0, "Servername", bold)
worksheet.write(row, 1, "Model", bold)
worksheet.write(row, 2, "Memory", bold)
worksheet.write(row, 3, "MAC address", bold)
worksheet.write(row, 4, "WWN address", bold)
worksheet.write(row, 5, "Status", bold)
row += 1

server_hardware = oneview_client.server_hardware.get_all()
#print(server_hardware)

for server in server_hardware:
 col = 0
 for ports in server['portMap']['deviceSlots']:
     for mac in ports['physicalPorts']: 
         for wwnn in mac['virtualPorts']:
             # print(server['name']+ " model:" + server['model'] + " " + str(server['memoryMb']) + " MB mac: " + str(mac['mac']) + " wwnn: " + str(wwnn['wwnn']))
             worksheet.write(row,col, server['name'])
             worksheet.write(row,col+1, server['model'])
             worksheet.write(row,col+2, server['memoryMb'])
             worksheet.write(row,col+3, mac['mac'])
             worksheet.write(row,col+4, wwnn['wwnn'])
             worksheet.write(row,col+5, server['status'])
             row += 1
workbook.close()
print ('Excel File roundtable.xlsx created')

ModuleNotFoundError: No module named 'xlsxwriter'